In [1]:
import time
import copy

from data import od, poi
from sklearn.preprocessing import MinMaxScaler
import math
scaler = MinMaxScaler()
from sklearn.metrics import mean_squared_error, r2_score
import ast  # safely evaluates string tuples like "(4, 5)" to actual tuples
import json

from models.divergence import Div
from sklearn.model_selection import KFold, GridSearchCV
from scipy.optimize import minimize
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D
import uuid
import base64
import pandas as pd  
from spatialgrid import SpatialGrid  
import numpy as np  
import matplotlib.pyplot as plt  
import sys  

from poidata import POIData  
from oddata import ODData
import os
import pandas as pd
import numpy as np

from transgm import TransGM
from dataset import DataSet
from divergence import Div


In [2]:
# Origin-Destination Data Processing Pipeline

# [1] - Assign OD data points into a unified grid

# Initialize ODData instances with spatial parameters
od_coventry = ODData(
    spacing_km=3,    # 3km grid spacing
    size=(8, 8)      # 8x8 target grid size
)

od_birmingham = ODData(
    spacing_km=4,    # 5km grid spacing  
    size=(8, 8)      # 8x8 target grid size
)

# Load OD trip data and POI data for each city
# Loads CSV files and processes POI categories
od_coventry.load_data('coventry')
od_birmingham.load_data('Birmingham')

# Extract coordinates from geometry strings in the loaded data
od_coventry.preprocess()
od_birmingham.preprocess()

# Generate origin-destination matrices with trip counts and spatial features
# Returns DataFrame with OD pairs, counts, distances, and POI features
df_od_birmingham = od_birmingham.create_od_matrix()
df_od_coventry = od_coventry.create_od_matrix()

Data shape: (36, 3)
dest_tensor shape: (8, 8, 11)
Best bandwidth: (0.0039692215962697985, 0.029058112211049716)
Data shape: (36, 3)
dest_tensor shape: (8, 8, 11)
Best bandwidth: (0.05556910234777718, 0.031479621561970525)
Data shape: (36, 3)
dest_tensor shape: (8, 8, 11)
Best bandwidth: (0.0039692215962697985, 0.029058112211049716)
Data shape: (36, 3)
dest_tensor shape: (8, 8, 11)
Best bandwidth: (0.0039692215962697985, 0.031479621561970525)
Data shape: (36, 3)
dest_tensor shape: (8, 8, 11)
Best bandwidth: (0.0039692215962697985, 0.029058112211049716)
Data shape: (36, 3)
dest_tensor shape: (8, 8, 11)
Best bandwidth: (0.0039692215962697985, 0.029058112211049716)
Data shape: (36, 3)
dest_tensor shape: (8, 8, 11)
Best bandwidth: (0.0039692215962697985, 0.029058112211049716)
Data shape: (36, 3)
dest_tensor shape: (8, 8, 11)
Best bandwidth: (0.0039692215962697985, 0.031479621561970525)
Data shape: (36, 3)
dest_tensor shape: (8, 8, 11)
Best bandwidth: (0.05556910234777718, 0.0266366028601289

In [ ]:
od_birmingham.categories

In [ ]:
# [2] Destination features
grid1 = od_birmingham.destinations[:,:,5]
grid2 = od_coventry.destinations[:,:,5]

In [ ]:
od_coventry.destinations[:,:,0]

In [ ]:
div = Div()
div.compute(grid1, grid2)

In [ ]:
results = {}
od_coventry.categories = sorted(od_coventry.categories)

i = 0
for c in od_coventry.categories:
    if c in ['trips', 'distance', 'origins', 'other', 'residential', 'public']:
        i += 1
        continue
    print(c)

    if c == 'leisure':
        c  = 'leisure'
    grid1 = od_coventry.destinations[:, :, i]
    grid2 = od_birmingham.destinations[:, :, i]

    jsd_div = div.compute(grid1, grid2)
    local_jsd,a,b = div.local_div_map(grid1, grid2)
    local_mean = np.mean(local_jsd)

    results[c] = {
        'feature': c,
        'observed_divergence': jsd_div,
    }

    results[c] = jsd_div


    i += 1
results

In [ ]:
# Plotting
plt.figure(figsize=(9, 3))
bars = plt.bar(list(results.keys()), list(results.values()), color='blue')
plt.xticks(rotation=45, ha='right')
plt.ylabel("Local Divergence")
# plt.title("Cosine Divergence per Feature with CV1 and CV2")
plt.ylim(0, 1.1)
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()


In [ ]:
# TODO: set dataset to the model
# unify the dataset before the div and the model
tgm = TransGM()

In [ ]:
bir_dataset = DataSet('birmingham')
cov_dataset = DataSet('coventry')

tgm.load_data(bir_dataset)
tgm.load_data(cov_dataset)

In [ ]:
cov_dataset.categories

In [ ]:
model = tgm.fit('birmingham', serial=1)

In [ ]:
model.results('base').to_dataframe().transpose()

In [ ]:
model.transfer('coventry').results('trans').to_dataframe().transpose()

In [ ]:
results

In [ ]:
list(results.values())

In [ ]:
model.with_divs(results).adapt('coventry').results('adapt').to_dataframe().transpose()

In [ ]:
# TODO: 
# 1. add flag for verbose
# 2. it seems base param_log is saved
# 3. done: calc div
# 4. plot some results
# 5. refactor adapt function
# 6. remove unused code
# 7. done: show how to create city grid!!!!! Example spatial_grid_od.ipynb in TranSIM

In [ ]:
test = DataSet('coventry')

In [ ]:
test.create_dist_matrix()